In [ ]:
!pip3 install langchain langchain_experimental python-decouple openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from decouple import config
import os
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# create LLM model
os.environ["OPENAI_API_KEY"] = "API KEY HERE"

OPENAI_API_KEY= os.getenv("API KEY HERE")
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# connect to our database
db = SQLDatabase.from_uri("sqlite:///chinook.db")

# create the database chain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

tools = [
    Tool(
        name="MathTool",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="Product_Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about products."
    )
]

# creating the agent
agent = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

# ask the LLM a question
agent.run("What is the post of Andrew Adams")



> Entering new AgentExecutor chain...

Invoking: `Product_Database` with `Andrew Adams`




> Entering new SQLDatabaseChain chain...
Andrew Adams
SQLQuery:SELECT "EmployeeId", "LastName", "FirstName", "Title", "ReportsTo", "BirthDate", "HireDate", "Address", "City", "State", "Country", "PostalCode", "Phone", "Fax", "Email" 
FROM employees 
WHERE "FirstName" = 'Andrew' AND "LastName" = 'Adams'
LIMIT 1;
SQLResult: [(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')]
Answer:General Manager Andrew Adams was born on February 18, 1962, and was hired on August 14, 2002. He can be reached at +1 (780) 428-9482 or andrew@chinookcorp.com.
> Finished chain.
General Manager Andrew Adams was born on February 18, 1962, and was hired on August 14, 2002. He can be reached at +1 (780) 428-9482 or andrew@chinookcorp.com.The post of 

'The post of Andrew Adams is General Manager.'